In [1]:
# Define your data directories
train_dir = 'COP_banknote/ds3/Train'
test_dir = 'COP_banknote/ds3/Test'
validation_dir = 'COP_banknote/ds3/Validation'

In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-10-21 20:42:41.827556: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-21 20:42:41.828789: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-21 20:42:41.847695: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-21 20:42:41.847708: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-21 20:42:41.847728: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [8]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Load the InceptionV3 model without the top layer (include_top=False)
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(12, activation='softmax')(x)  # Use 'softmax' for multi-class classification

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers from the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generators
batch_size = 32
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),  # InceptionV3 input size
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    classes=['100k', '100kf', '10k', '10kf', '50k', '50kf', '5k', '5kf', '20k', '20kf', '2k', '2kf']
)

# Validation data generator
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    classes=['100k', '100kf', '10k', '10kf', '50k', '50kf', '5k', '5kf', '20k', '20kf', '2k', '2kf']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=3,  # You can adjust the number of epochs
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Evaluate on test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    classes=['100k', '100kf', '10k', '10kf', '50k', '50kf', '5k', '5kf', '20k', '20kf', '2k', '2kf']
)

test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f'InceptionV3 Test accuracy: {test_acc * 100:.2f}%')

# Save the trained model for later use
model.save('banknote_inceptionv3_multiclass.h5')


Found 8600 images belonging to 12 classes.
Found 2913 images belonging to 12 classes.
Epoch 1/3
269/269 [==============================] - 204s 751ms/step - loss: 0.7224 - accuracy: 0.7816 - val_loss: 0.2562 - val_accuracy: 0.9156
Epoch 2/3
269/269 [==============================] - 189s 701ms/step - loss: 0.2154 - accuracy: 0.9291 - val_loss: 0.1800 - val_accuracy: 0.9389
Epoch 3/3
269/269 [==============================] - 165s 613ms/step - loss: 0.1662 - accuracy: 0.9426 - val_loss: 0.0397 - val_accuracy: 0.9907
Found 2903 images belonging to 12 classes.
91/91 [==============================] - 19s 205ms/step - loss: 0.0397 - accuracy: 0.9945
InceptionV3 Test accuracy: 99.45%
